In [ ]:
import oracledb

CFG_DIR  = ""          
USER     = ""
PASSWORD = ""
DSN      = ""      

conn = oracledb.connect(
    user=USER, password=PASSWORD, dsn=DSN,
    config_dir=CFG_DIR,
    wallet_location=CFG_DIR,
    wallet_password=PASSWORD,  
)

with conn.cursor() as c:
    c.execute("select banner_full from v$version")
    print(*c.fetchall(), sep="\n")

    c.execute("BEGIN EXECUTE IMMEDIATE 'CREATE TABLE VEC_PROBE (id NUMBER, v VECTOR(4))'; END;")
    c.execute("BEGIN EXECUTE IMMEDIATE 'DROP TABLE VEC_PROBE PURGE'; END;")
    print("OK: VECTOR disponível")

conn.close()
print("Conexão OK")


('Oracle Database 23ai Enterprise Edition Release 23.0.0.0.0 - for Oracle Cloud and Engineered Systems\nVersion 23.9.0.25.08',)
OK: VECTOR disponível
Conexão OK


In [ ]:
from purecpp_oracledb.vectordb import make_backend, Document

CFG_DIR = ""

cfg = {
    "user": "",
    "password": "",
    "dsn": "",
    "config_dir": CFG_DIR,
    "wallet_location": CFG_DIR,
    "wallet_password": "",

    "table": "",
    "dim": 4,
    "metric": "COSINE",
    "ensure_schema": False,                    
    "index_algorithm": "IVF",
    "index_params": "type IVF, neighbor partitions 256",
    "target_accuracy": 95,
    "debug": True,
}

bk = make_backend("oracle", cfg)

bk.insert([
    Document("primeiro texto", [0.1, 0.2, 0.3, 0.4], {"lang": "pt"}),
    Document("segundo texto",  [0.9, 0.1, 0.0, 0.2], {"lang": "en"}),
])

print("=== query ===")
for r in bk.query([0.1, 0.2, 0.25, 0.35], k=2, filter={"lang": "pt"}):
    print(f"score={r.score:.6f} meta={r.doc.metadata} page='{r.doc.page_content}'")

bk.close()


[OracleBackend] connect attempt 1: {'user': 'ADMIN', 'password': '***', 'dsn': 'XU62UQK9OEAD0789_high', 'ssl_server_dn_match': True, 'config_dir': '/Users/diogovieira/Documents/oracle/ai', 'wallet_location': '/Users/diogovieira/Documents/oracle/ai', 'wallet_password': '***'}
=== query ===
score=0.001952 meta={'lang': 'pt', 'id': 'ae722c44-ef1e-403a-822f-aef64dc50616'} page='primeiro texto'
score=0.001952 meta={'lang': 'pt', 'id': 'bad44260-8720-43db-98ea-757894ae632b'} page='primeiro texto'
